# Modele Seq2Seq i atencja
Poniższy notebook jest inspirowany tym tutorialem PyTorcha: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html. Zachęcam żeby zajrzeć tam po więcej informacji.

W tym notebooku będziemy próbować rozwiązać problem automatycznego tłumaczenia zdań z jednego języka naturalnego na drugi -- konkretniej z języka polskiego na angielski. Dla przykładu model otrzymujący zdanie:

> Myślę, że mnie okłamałeś

Powinien zwrócić zdanie
> I think you lied to me.



# Importy i przygotowanie danych
Poniżej znajdują się importy bibliotek potrzebnych do rozwiązania problemu a także skrypt do ładowania zbioru danych zawierającego pary zdań w języku polskim i angielskim. Poniższy kod można odpalić i schować, ale zachęcamy do zaznajomienia się z tym jak wygląda obróbka danych.


In [59]:
%matplotlib inline

In [60]:
from __future__ import unicode_literals, print_function, division
from io import open
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import unicodedata
import string
import numpy as np
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [61]:
!wget https://raw.githubusercontent.com/gmum/ml2020-21/master/lab/resources/pol.txt

--2021-01-27 11:33:30--  https://raw.githubusercontent.com/gmum/ml2020-21/master/lab/resources/pol.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5865362 (5.6M) [text/plain]
Saving to: ‘pol.txt.3’

pol.txt.3           100%[===================>]   5.59M  --.-KB/s    in 0.07s   

2021-01-27 11:33:30 (77.4 MB/s) - ‘pol.txt.3’ saved [5865362/5865362]



Poniżej przygotowujemy klasę `Lang` która będzie służyła jako struktura do obsługiwania naszego języka (osobna dla angielskiego i polskiego w naszym przypadku). Do każdego słowa w języku przypisujemy indeks (liczbę porządkową identyfikującą słowo). Dodatkowo definiujemy trzy dodatkowe indeksy:

* 0 dla początku zdania (Start of Sentence, SOS)
* 1 dla końca zdania (End of Sentence, EOS)
* 2 dla paddingu ("pustych" wartości). Wartościami tymi będziemy wypełniać zdania w batchu tak, żeby wszystkie były równej długości -- dzięki temu łatwiej będzie zrównoleglić przetwarzanie ich na GPU.

In [62]:


SOS_token = 0
EOS_token = 1
PAD_token = 2

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS", 2: "PAD"}
        self.n_words = 3 # Count SOS, EOS and PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Funkcje do normalizowania wchodzących zdań - zamieniamy Unicode na ASCII, zamieniamy wszystkie wielkie litery na małe itd.

In [63]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = s.replace("ł", "l")
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(f'{lang1}.txt', encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Wyrzućmy zdania które są zbyt długie (ponad 20 słów).

In [64]:
MAX_LENGTH = 20

def filterPair(p):
    return len(p[0].split(' ')) <= MAX_LENGTH and \
        len(p[1].split(' ')) <= MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Pełny proces przetwarzania danych wygląda następująco:

- Wczytujemy plik z danymi, dzielimy go na pary zdań.
- Normalizujemy tekst
- Zamieniamy zdania w listy słów.




In [65]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print(pairs[0])
    print(f"Read {len(pairs)} sentence pairs")
    pairs = filterPairs(pairs)
    print(f"Trimmed to {len(pairs)} sentence pairs")
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('pol', 'eng', True)
print("Przykładowe pary zdań:")
for _ in range(3):
    print(random.choice(pairs))

Reading lines...
['idz .', 'go .']
Read 40389 sentence pairs
Trimmed to 40289 sentence pairs
Counting words...
Counted words:
eng 20475
pol 8084
Przykładowe pary zdań:
['glod najlepszym kucharzem .', 'hunger is the best sauce .']
['ta rozmowa nic nam nie da .', 'that kind of talk will get you nowhere .']
['zamierzam tam pojsc .', 'i intend to go there .']


Na koniec definiujemy jeszcze funkcje, które pozwolą nam zamienić zdania w tensory, które nasza sieć będzie w stanie zrozumieć.

In [66]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


def pad_sequences(data_batch):
    pl_batch, en_batch = [], []
    for pl_sentence, en_sentence in data_batch:
        pl_batch += [pl_sentence]
        en_batch += [en_sentence]
    pl_batch = pad_sequence(pl_batch, padding_value=PAD_token, batch_first=True)
    en_batch = pad_sequence(en_batch, padding_value=PAD_token, batch_first=True)
    return pl_batch, en_batch

def prepare_dataset(batch_size):
    rng = np.random.RandomState(567)
    indices = np.arange(len(pairs))
    rng.shuffle(indices)
    train_indices = indices[:int(len(pairs) * 0.8)]
    test_indices = indices[int(len(pairs) * 0.8):]
    train_pairs = list(pairs[idx] for idx in train_indices)
    test_pairs = list(pairs[idx] for idx in test_indices)
    tensor_train_pairs = [tensorsFromPair(pairs[idx]) for idx in train_indices]
    tensor_test_pairs = [tensorsFromPair(pairs[idx]) for idx in test_indices]

    train_loader = DataLoader(tensor_train_pairs, batch_size=batch_size,
                            shuffle=True, collate_fn=pad_sequences)
    test_loader = DataLoader(tensor_test_pairs, batch_size=batch_size,
                            shuffle=True, collate_fn=pad_sequences)
    return train_pairs, test_pairs, train_loader, test_loader

# Model Seq2Seq



W tym celu wykorzystany rekurencyjne sieci neuronowe (RNN-y), które poznaliśmy na poprzednich zajęciach. Konkretniej zbudujemy za ich pomocą model Sequence to Sequence (Seq2Seq), w której wykorzystamy dwie sieci rekurencyjne:
1. Enkoder, który będzie przyjmował kolejno słowa ze zdania wejściowego i kompresował informacje o nich w swoim stanie ukrytym.
2. Dekoder, który będzie generował kolejne słowa w języku docelowym. 

![seq2seq](https://docs.chainer.org/en/stable/_images/seq2seq.png)
Źródło: https://docs.chainer.org/

## Funkcje pomocnicze i ewaluacyjne

In [67]:
def predict(encoder, decoder, inputs, targets=None, max_len=MAX_LENGTH):
    batch_size = inputs.size(0)

    encoder_outputs, encoder_hidden = encoder(inputs)

    decoder_input = torch.tensor([[SOS_token]] * batch_size, device=device)
    decoder_hidden = encoder_hidden
    decoder_output, decoder_attention = decoder(
        decoder_input,
        decoder_hidden,
        targets=targets,
        max_len=max_len,
        encoder_outputs=encoder_outputs)
    return decoder_output, decoder_attention

def translate(encoder, decoder, sentence):
    inputs = tensorFromSentence(input_lang, sentence).unsqueeze(0).cuda()
    decoder_output, decoder_attention = predict(encoder, decoder, inputs)

    decoded_words = []
    for word in decoder_output[0]:
        top_word = word.argmax(-1).item()
        decoded_words.append(output_lang.index2word[top_word])
        if top_word == EOS_token:
            break

    if decoder_attention is not None:
        # [out_words, in_words]
        att = decoder_attention.cpu().detach().numpy()
        att = att[0, :len(decoded_words), :]
        fig, ax = plt.subplots()

        ax.imshow(att)
        ax.set_xticklabels([''] + sentence.split(' ') +
                        ['EOS'], rotation=90)
        ax.set_yticklabels([''] + decoded_words)


        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

        plt.show()

    return decoded_words
        
def translate_randomly(encoder, decoder, pairs, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = translate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Pętla trenująca

In [68]:
def train(encoder, decoder, lr=0.01, batch_size=256, teacher_forcing_ratio=0.5, epochs_num=100):

    # Prepare dataset, loss functions, optimizer
    train_pairs, test_pairs, train_loader, test_loader = prepare_dataset(batch_size)

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_token)

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=lr)

    encoder.cuda()
    decoder.cuda()

    for epoch in range(epochs_num + 1):

        # Training
        epoch_train_loss = 0.
        for in_batch, out_batch in train_loader:
            in_batch, out_batch = in_batch.cuda(), out_batch.cuda()

            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
        
            teacher_inputs = out_batch if random.random() < teacher_forcing_ratio else None
        
            decoder_output, decoded_attention = predict(
                encoder, decoder, in_batch,
                targets=teacher_inputs,
                max_len=out_batch.size(1)
            )
        
            loss = criterion(decoder_output.transpose(1, 2), out_batch)
            loss.backward()
        
            encoder_optimizer.step()
            decoder_optimizer.step()

            epoch_train_loss += loss.item()

        # Evaluation
        if epoch % 25 == 0:
            with torch.no_grad():
                print("=" * 25, "Translation test", "=" * 25)
                translate_randomly(encoder, decoder, test_pairs, n=5)

        mean_train_loss = epoch_train_loss / len(train_loader)
        print(f"Epoch: {epoch}. Train loss: {mean_train_loss}")

# Zadanie 1 - Dekoder w Seq2Seq (4 pkt.)

W tym zadaniu należy zaimplementować dekoder z modelu Seq2Seq. Kod enkodera jest dostępny poniżej i ma Państwu ułatwić odpowiednie zaimplementowanie dekodera.


Dekoder otrzymuje na wejściu następujące argumenty:
- `input` - tensor o wymiarach `[batch_size, 1]`  zawierający tokeny SOS oznaczające początek zdania (<BOS> na obrazku).
- `targets` - `None` albo `torch.tensor` o wymiarach `[batch_size, seq_len]` zawierający indeksy słów w języku docelowym. Jeżeli jest podany to należy zaimplementować teacher forcing na jego podstawie.
- `max_len` - Długość sekwencji, którą mamy zwrócić.
- `encoder_outputs` - w tym zadaniu ten argument należy zignorować, przyda się dopiero w kolejnym zadaniu.

Dekoder ma zwrócić dwie zmienne:
- `output` - tensor o wymiarach `[batch_size, seq_len, vocab_size]` reprezentujące logity, które po zaaplikowaniu softmaksa (co będzie zrobione już poza dekoderem) będą reprezentowały prawdopodobieństwa słów przewidzianych przez nasz dekoder.
- `attention_weights` - w tym zadaniu należy zawsze zwracać `None`.


**HINT 1**: Warto pamiętać o argumencie `batch_first=True` przy definiowaniu RNN-a.

**HINT 2**: W enkoderze mogliśmy użyć jednego wywołania klasy LSTM, jako że od razu mieliśmy wszystkie wejścia (słowa języka wejściowego). W przypadku dekodera nie jest to możliwe, jako że wejściem w kroku `t+1` jest wyjście z kroku `t`. Oznacza to że prawdopodobnie potrzebna będzie pętla `for`.

In [69]:
class EncoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)

    def forward(self, input):
        embedded = self.embedding(input)
        output, hidden = self.lstm(embedded)
        return output, hidden

In [70]:
class DecoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(vocab_size, embedding_size) 
        self.lstm = nn.LSTM(embedding_size, hidden_size, batch_first=True)
        self.lin = nn.Linear(hidden_size, vocab_size)

    def forward(self, input, hidden, targets=None, max_len=None, encoder_outputs=None):
        result = None
        for i in range(max_len):
            embedded = self.embedding(input if targets is None else targets[:, i].unsqueeze(1))
            output, hidden = self.lstm(embedded, hidden)
            output = self.lin(output)
            result = output if result is None else torch.cat((result, output), dim=1)
            input = nn.Softmax(dim=2)(output).argmax(dim=2)
        return result, None

In [71]:
hidden_size = 128
embedding_size = 256
encoder = EncoderRNN(input_lang.n_words, embedding_size, hidden_size).to(device)
decoder = DecoderRNN(output_lang.n_words, embedding_size, hidden_size).to(device)

train(encoder, decoder, lr=0.005, epochs_num=100)

========================= Translation test =========================
> tomek jest wyzszy od wszystkich innych dzieci w klasie .
= tom is taller than any of the other kids in his class .
< i i . . . . . . . . . . . . . . . . . .

> kioto ma duzo uniwersytetow .
= kyoto has many universities .
< i i . . . . . . . . . . . . . . . . . .

> kto odpowiedzial ?
= who responded ?
< i i . . . . . . . . . . . . . . . . . .

> musisz wziac to pod uwage .
= you have to take that into account .
< i i . . . . . . . . . . . . . . . . . .

> powiedz nam wiecej .
= tell us more .
< i i . . . . . . . . . . . . . . . . . .

Epoch: 0. Train loss: 4.323974558285305
Epoch: 1. Train loss: 3.0649927490287356
Epoch: 2. Train loss: 2.7024922891268655
Epoch: 3. Train loss: 2.434321341888299
Epoch: 4. Train loss: 2.197949752329834
Epoch: 5. Train loss: 2.751747729286315
Epoch: 6. Train loss: 2.5202347403540024
Epoch: 7. Train loss: 2.105283638225898
Epoch: 8. Train loss: 2.084834266217455
Epoch: 9. Train loss: 2.

# Zadanie 2 - Atencja w RNN-ach (3 pkt.)

![seq2seq z dekoderem](https://www.researchgate.net/profile/Chandan_Reddy6/publication/329464533/figure/fig3/AS:701043021197314@1544153089772/An-attention-based-seq2seq-model.ppm)

Źródło: https://github.com/google/seq2seq

W tym zadaniu należy napisać kod nowego dekodera, który ma działać podobnie jak dekoder w poprzednim zadaniu, ale jednocześnie ma wykorzystywać mechanizm atencji.

W normalnym dekoderze, w kroku `t` wejściem do komórki LSTM-a (pomijamy tutaj przekazywanie stanu ukrytego) była wyłącznie zembeddowana reprezentacja $\bar{y}_t$. W dekoderze z atencją na wejściu podawna będzie konkatenacja tego wektora oraz specjalnego wektora $z_t$ stworzonego na podstawie wyjść z enkodera: $\tilde{h}_t = [\bar{y}_t, z_t]$. 

Wektor $z_t$ jest pozyskiwany za pomocą mechanizmu atencji. Intuicyjnie chcielibyśmy w nim zebrać informacje z enkodera, które będą najistotniejsze przy dekodowaniu aktualnego słowa. Przyjmijmy, że mamy funkcję alignmentu $a(h, e)$, która jest nam w stanie powiedzieć jak bardzo podobne do siebie są stan ukryty dekodera $h$ oraz reprezentacja enkodera dla słowa $e$.

Wtedy 

$$
\begin{align}
w_i &= \frac{ \exp(a(h, e_i)) }{\sum_{j} \exp(a(h, e_j))} \\
z_t &= \sum_i e_i \cdot w_i
\end{align}
$$


W naszym wypadku funkcja alignmentu $a(h, e)$ ma być siecią neuronową z dwoma warstwami o kolejnych wymiarach: `[2 * hidden_size, hidden_size, 1]` i aktywacją tanh po pierwszej warstwie.

Argumenty wejściowe i wyjściowe z dekodera są takie same jak w poprzednim z zadaniu z wyjątkiem:
- Tym razem na wejściu otrzymujemy tensor `encoder_outputs` o wymiarach `[batch_size, encoder_seq_len, hidden_size]`. To są reprezentacje $e_i$, które należy wykorzystać w mechanizmie atencji.
- Tym razem na wyjściu `attention_weights` powinno być tensorem o wymiarach `[batch_size, decoder_seq_len, encoder_seq_len]` zawierającym wagi $w_i$. **HINT:** wartości tego tensora powinny się sumować do jedynki na ostatnim wymiarze.

In [77]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(AttnDecoderRNN, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size + hidden_size, hidden_size, batch_first=True)
        self.lin = nn.Linear(hidden_size, vocab_size)
        self.alignment = nn.Sequential(
            nn.Linear(2 * hidden_size, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, 1)
        )

    def forward(self, input, hidden, targets=None, max_len=None, encoder_outputs=None):
        result = None
        seq_att_weights = None
        for i in range(max_len):
            stacked = torch.stack([hidden[0].squeeze(0)] * encoder_outputs.shape[1], dim=1)
            att_scores = self.alignment(torch.cat((stacked, encoder_outputs), dim=2))
            att_weights = nn.Softmax(dim=1)(att_scores)
            att_output = torch.sum(att_weights * encoder_outputs, dim=1)
            
            embedded = self.embedding(input if targets is None else targets[:, i].unsqueeze(1))
            output, hidden = self.lstm(torch.cat((embedded, att_output.unsqueeze(1)), dim=2), hidden)
            output = self.lin(output)
            result = output if result is None else torch.cat((result, output), dim=1)
            input = nn.Softmax(dim=2)(output).argmax(dim=2)
        return result, seq_att_weights

In [78]:
hidden_size = 128
embedding_size = 256
encoder = EncoderRNN(input_lang.n_words, embedding_size, hidden_size).to(device)
decoder = AttnDecoderRNN(output_lang.n_words, embedding_size, hidden_size).to(device)

train(encoder, decoder, lr=0.005, epochs_num=100)

========================= Translation test =========================
> tom polozyl pokrywe z powrotem na smietniku .
= tom put the lid back on the trash can .
< tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom

> spodziewam sie otrzymac dzis emaila od firmy .
= i expect to receive an email from the company today .
< tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom

> uswiadomilem sobie ze powinienem uczyc sie francuskiego nieco intensywniej .
= i realized i needed to study french a little harder .
< i is . . . . . . . . . . . . . . . . . .

> banany sa zolte .
= bananas are yellow .
< tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom

> studenci sa na wakacjach .
= the students are on vacation .
< i the the the the the the the the the the the the the the the the the the the

Epoch: 0. Train loss: 4.259778755051749
Epoch: 1. Train loss: 3.423019192994587
Epoch: 2. Train loss: 2.7167509482020424
E

# Powiązana literatura

* [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
* [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)

## Przydatne tutoriale

* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
* https://github.com/bentrevett/pytorch-seq2seq
* https://github.com/gmum/AppliedDL2020/tree/master/Week%207 - materiały z kursu Applied Deep Learning prowadzonego w semestrze letnim


# Zadanie dodatkowe: Transformer (7 pkt.)

Na podstawie pracy [Attention is All You Need](https://arxiv.org/abs/1706.03762) oraz strony [The Annotated Transformer](http://nlp.seas.harvard.edu/2018/04/03/attention) zaimplementuj transformera działającego na powyższym zadaniu tłumaczenia z polskiego na angielski.

![transformer](http://nlp.seas.harvard.edu/images/the-annotated-transformer_14_0.png)